<a href="https://colab.research.google.com/github/AdrienAudouard/Football-deep-learning/blob/master/Football_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import des librairies


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
!pip install category_encoders

In [ ]:
tf.__version__

'2.3.0'

# Importing Dataset

In [ ]:
dataset = pd.read_csv('export.csv')
x = dataset.iloc[:, [True, True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]].values # Toutes les colonnes à partir de l'index 3 jusqu'à l'avant dernière 
y = dataset.iloc[:, 5].values # Uniquement la dernière colonne du dataset (expected)

In [ ]:
print(x[0])

['Guingamp' 'Monaco' 'French Ligue 1' '2016/17' 1 1 0 0 0 0 0 0 0 0 0 0 3
 2 2 0 0 0 0 0 0 0 0 0 0 3]


In [ ]:
print(y)

['HOME' 'HOME' 'AWAY' ... 'AWAY' 'HOME' 'HOME']


# Encoding Data

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import category_encoders as ce

ct = ColumnTransformer(transformers=[('encoder', ce.BinaryEncoder(), [0, 1, 2, 3])], remainder='passthrough')
cty = ColumnTransformer(transformers=[('encoder', ce.BinaryEncoder(), [0])], remainder='passthrough')

x = np.array(ct.fit_transform(x))

y=y.reshape(-1,1)

y = np.array(cty.fit_transform(y))


/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
print(y)

[[0 0 1]
 [0 0 1]
 [0 1 0]
 ...
 [0 1 0]
 [0 0 1]
 [0 0 1]]


In [ ]:
print(x)

[[0 0 0 ... 0 0 3]
 [0 0 0 ... 0 0 3]
 [0 0 0 ... 0 0 3]
 ...
 [0 0 0 ... 6 4 7]
 [0 0 0 ... 6 1 6]
 [0 0 1 ... 1 2 7]]


# Splitting the dataset into the Training set and Test set


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

Feature Scaling

In [ ]:
 from sklearn.preprocessing import StandardScaler
 sc = StandardScaler()
 x_train = sc.fit_transform(x_train)
 x_test = sc.transform(x_test)

# Init ANN

In [ ]:
ann = tf.keras.models.Sequential()

### Add layers


In [ ]:
ann.add(tf.keras.layers.Dense(64, activation='sigmoid')) 
ann.add(tf.keras.layers.Dense(32, activation='linear')) 

### Output Layers

In [ ]:
ann.add(tf.keras.layers.Dense(units = 3, activation='softmax')) 

### Compile

In [ ]:
ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training

In [ ]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 100, validation_data=(x_test, y_test))

Epoch 1/100
231/231 [==============================] - 1s 3ms/step - loss: 0.6690 - accuracy: 0.6396 - val_loss: 0.6381 - val_accuracy: 0.6634
Epoch 2/100
231/231 [==============================] - 0s 2ms/step - loss: 0.6450 - accuracy: 0.6508 - val_loss: 0.6396 - val_accuracy: 0.6602
Epoch 3/100
231/231 [==============================] - 0s 2ms/step - loss: 0.6518 - accuracy: 0.6486 - val_loss: 0.6508 - val_accuracy: 0.6401
Epoch 4/100
231/231 [==============================] - 0s 2ms/step - loss: 0.6717 - accuracy: 0.6317 - val_loss: 0.7364 - val_accuracy: 0.6179
Epoch 5/100
231/231 [==============================] - 0s 2ms/step - loss: 0.6716 - accuracy: 0.6339 - val_loss: 0.6427 - val_accuracy: 0.6645
Epoch 6/100
231/231 [==============================] - 0s 2ms/step - loss: 0.6854 - accuracy: 0.6190 - val_loss: 0.9366 - val_accuracy: 0.4656
Epoch 7/100
231/231 [==============================] - 0s 2ms/step - loss: 0.6973 - accuracy: 0.6160 - val_loss: 0.6434 - val_accuracy: 0.6526

### Predict

In [ ]:
y_pred = ann.predict(x_test)

In [ ]:
print(y)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)